In [87]:
# run once
# ! pip install gspread oauth2client df2gspread boto3
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd

my_key = get_key('koray_data')

all_users = ff_utils.search_metadata('/search/?type=User', key = my_key)
all_labs = ff_utils.search_metadata('/search/?type=Lab', key = my_key)
print(len(all_users))
print(len(all_labs))
# all_labs = 
# all_awards = 

440
86


In [89]:
print(all_labs[0])

{'display_title': 'Karen Adelman, HARVARD', 'link_id': '~labs~karen-adelman-lab~', 'postal_code': '', 'institute_name': '', 'address1': '', 'country': '', 'fax': '', 'city': '', 'institute_label': 'HARVARD', 'external_references': [], 'state': '', 'submitted_by': {'link_id': '~users~56c9c683-bb11-471b-b590-c656f7dc03c1~', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'userid.56c9c683-bb11-471b-b590-c656f7dc03c1'], 'view': ['group.admin', 'group.read-only-admin', 'remoteuser.EMBED', 'remoteuser.INDEXER', 'userid.56c9c683-bb11-471b-b590-c656f7dc03c1']}, '@id': '/users/56c9c683-bb11-471b-b590-c656f7dc03c1/', 'display_title': 'Luisa Mercado', 'uuid': '56c9c683-bb11-471b-b590-c656f7dc03c1'}, 'uuid': '0aae4b45-b6de-45da-84e6-738667c36f03', 'status': 'current', 'name': 'karen-adelman-lab', 'date_created': '2018-12-10T16:44:02.075461+00:00', '@type': ['Lab', 'Item'], 'last_modified': {'modified_by': {'link_id': '~users~56c9c683-bb11-471b-b590-c656f7dc03c1~', 'pri

In [90]:

def compare(row, columns):
    extra = ""
    second_mail = ""
    delete = False
    first_name = row['First Name'].strip()
    last_name = row['Last Name'].strip()
    mail = row['Email'].strip().lower()
    role = row['Role'].strip()
    
    if not first_name:
        return
    try:
        contact_pi = row['PI Contact']
    except:
        contact_pi = row['Contact PI']
    if "Additional email" in columns:
        second_mail = row['Additional email']
    
    for val in row:
        if 'delete' in val.lower():
            delete = True
        elif 'remove' in val.lower():
            delete = True
    
    # Check for first name last name combination
    res1 = [i for i in all_users if i['first_name'] == first_name if i['last_name'] == last_name]
    if res1:
        if delete:
            print(mail, 'user exists but tagged with delete')
        return True
    
    # Are the name surname reversed
    res2 = [i for i in all_users if i['first_name'] == last_name if i['last_name'] == first_name]
    if res2:
        print(mail, 'user has reversed name surname')
        if delete:
            print(mail, 'user exists but tagged with delete')
        return True
    
    # Do we have the email 
    res3 = [i for i in all_users if i['email'] == mail]
    if res3:
        #print(mail, 'found on our system, OH name is',first_name, last_name, 'DATA name is', res3[0]['first_name'], res3[0]['last_name'])
        if delete:
            print(mail, 'user exists but tagged with delete')
        return True
    
    if second_mail:
        res4 = [i for i in all_users if i['email'] == second_mail]
        if res4:
            #print(mail, 'found on our system, OH name is',first_name, last_name, 'DATA name is', res3[0]['first_name'], res3[0]['last_name'])
            if delete:
                print(mail, 'user exists but tagged with delete')
            return True
    
    # Do we have same surname
    res5 = [i for i in all_users if i['last_name'] == last_name]
    f_names = [i['first_name'] for i in res5]
    if res5:
        if delete:
            return
        print(mail, first_name, last_name, 'matches by surname to these names', ', '.join(f_names))
        return True
    
    # if we cant find but tagged with delete
    if delete:
        return
    
    if not mail:
        return
    
    
    
    print(first_name, last_name, mail)
        
    
            
            


In [86]:
# GET OH SHEET
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json

# GET KEY FROM S3 
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

# The ID and range of a sample spreadsheet.
book_id = '13gFjHcYEwYL2Xpr9-IHtJNYYnYlAQVdyXvCUeSzla6w'
sheet_names = ['DCIC & OH', 'IT', 'NT', 'NOFIC', 'NBC']
book = gc.open_by_key(book_id)

for name in sheet_names:
    print()
    print(name)

    worksheet = book.worksheet(name)
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    col_names = df.columns.values

    for index, row in df.iterrows():
        compare(row, col_names)



DCIC & OH
Connor Horton connorhorton@college.harvard.edu
Yiran Hou 
joshhjang@gmail.com user exists but tagged with delete
Erica C. Pehrsson 
ananda.datta@wustl.edu user exists but tagged with delete
Irene Hom Jacobo ihjacobo@eng.ucsd.edu
Carol Kling ckling@ucsd.edu
Kathia Zaleta Rivera kzaletar@eng.ucsd.edu
wec125@eng.ucsd.edu Weizhong Chen matches by surname to these names Yu, Hongtao, Lin
Lucie Hebert luhebert@eng.ucsd.edu
hzzheng@eng.ucsd.edu Haizi Zheng matches by surname to these names Qinsi
Weixin Wu wew245@eng.ucsd.edu
Kate Rivera car032@eng.ucsd.edu

IT
Frank Chuang frank.chuang@cbst.ucdavis.edu
 Ruiwu Liu matches by surname to these names Jie, Tristin
dengw@berkeley.edu Jorge Deng matches by surname to these names Wulan, Xinxian
Astou Robles arobles77@berkeley.edu
Frank Xie frank.l.xie@berkeley.edu
Maximillian Huisman maximiliaan.huisman@umassmed.edu
Greg Cadwell gcadwell@salk.edu
Preston Dennett pdennett@salk.edu

NT
Thomas Gilgenast tgg@seas.upenn.edu
Jon Beagan jbeagan@se